# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [3]:
spaceship.drop_duplicates()
spaceship.dropna()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [5]:
spaceship

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [6]:
spaceship.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [7]:
mask = (spaceship.applymap(lambda x: isinstance(x, float) and x == 0.0))
mask

C:\Users\ostor\AppData\Local\Temp\ipykernel_20404\2721159345.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = (spaceship.applymap(lambda x: isinstance(x, float) and x == 0.0))


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,False,False,False,False,False,False,False,True,True,True,True,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,False,False,False,False,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,False,False,False,False,False,False,True,False,True,False,False,False,False
8689,False,False,False,False,False,False,False,True,True,True,True,True,False,False
8690,False,False,False,False,False,False,False,True,True,False,False,True,False,False
8691,False,False,False,False,False,False,False,True,False,True,False,False,False,False


In [8]:
spaceship_cleaned=spaceship[~mask.any(axis=1)]
spaceship_cleaned


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
11,0008_03,Europa,False,B/1/P,55 Cancri e,45.0,False,39.0,7295.0,589.0,110.0,124.0,Wezena Flatic,True
14,0011_01,Earth,False,F/2/P,TRAPPIST-1e,28.0,False,8.0,974.0,12.0,2.0,7.0,Elle Bertsontry,True
54,0056_03,Europa,False,A/1/S,TRAPPIST-1e,27.0,False,279.0,605.0,3374.0,286.0,3.0,Zinoces Tractive,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8618,9196_02,Europa,False,C/339/S,TRAPPIST-1e,31.0,False,1.0,104.0,1338.0,698.0,126.0,Majorab Poustive,True
8619,9196_03,Europa,False,C/339/S,TRAPPIST-1e,23.0,False,381.0,794.0,52.0,654.0,1317.0,Chellus Poustive,False
8626,9202_01,Europa,False,C/309/P,TRAPPIST-1e,26.0,False,8586.0,77.0,114.0,29.0,4.0,Alark Eguing,False
8657,9234_01,Earth,False,G/1500/P,TRAPPIST-1e,49.0,False,6.0,1.0,638.0,1107.0,23.0,Catha Bushins,False


In [9]:
spaceship_cleaned_2 = spaceship_cleaned.reset_index(drop=True)
spaceship_cleaned_2

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
1,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
2,0008_03,Europa,False,B/1/P,55 Cancri e,45.0,False,39.0,7295.0,589.0,110.0,124.0,Wezena Flatic,True
3,0011_01,Earth,False,F/2/P,TRAPPIST-1e,28.0,False,8.0,974.0,12.0,2.0,7.0,Elle Bertsontry,True
4,0056_03,Europa,False,A/1/S,TRAPPIST-1e,27.0,False,279.0,605.0,3374.0,286.0,3.0,Zinoces Tractive,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,9196_02,Europa,False,C/339/S,TRAPPIST-1e,31.0,False,1.0,104.0,1338.0,698.0,126.0,Majorab Poustive,True
307,9196_03,Europa,False,C/339/S,TRAPPIST-1e,23.0,False,381.0,794.0,52.0,654.0,1317.0,Chellus Poustive,False
308,9202_01,Europa,False,C/309/P,TRAPPIST-1e,26.0,False,8586.0,77.0,114.0,29.0,4.0,Alark Eguing,False
309,9234_01,Earth,False,G/1500/P,TRAPPIST-1e,49.0,False,6.0,1.0,638.0,1107.0,23.0,Catha Bushins,False


In [10]:
spaceship_cleaned_2.nunique()

PassengerId     311
HomePlanet        3
CryoSleep         1
Cabin           299
Destination       3
Age              53
VIP               2
RoomService     208
FoodCourt       238
ShoppingMall    187
Spa             208
VRDeck          214
Name            303
Transported       2
dtype: int64

In [13]:
import pandas as pd

# Define the mapping for each categorical column
home_planet_mapping = {value: idx for idx, value in enumerate(spaceship_cleaned_2['HomePlanet'].unique())}
destination_mapping = {value: idx for idx, value in enumerate(spaceship_cleaned_2['Destination'].unique())}
vip_mapping = {value: idx for idx, value in enumerate(spaceship_cleaned_2['VIP'].unique())}

# Apply the mapping
spaceship_cleaned_2['HomePlanet'] = spaceship_cleaned_2['HomePlanet'].map(home_planet_mapping)
spaceship_cleaned_2['Destination'] = spaceship_cleaned_2['Destination'].map(destination_mapping)
spaceship_cleaned_2['VIP'] = spaceship_cleaned_2['VIP'].map(vip_mapping)

spaceship_cleaned_2

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0002_01,0,False,F/0/S,0,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
1,0004_01,0,False,F/1/S,0,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
2,0008_03,1,False,B/1/P,1,45.0,0,39.0,7295.0,589.0,110.0,124.0,Wezena Flatic,True
3,0011_01,0,False,F/2/P,0,28.0,0,8.0,974.0,12.0,2.0,7.0,Elle Bertsontry,True
4,0056_03,1,False,A/1/S,0,27.0,0,279.0,605.0,3374.0,286.0,3.0,Zinoces Tractive,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,9196_02,1,False,C/339/S,0,31.0,0,1.0,104.0,1338.0,698.0,126.0,Majorab Poustive,True
307,9196_03,1,False,C/339/S,0,23.0,0,381.0,794.0,52.0,654.0,1317.0,Chellus Poustive,False
308,9202_01,1,False,C/309/P,0,26.0,0,8586.0,77.0,114.0,29.0,4.0,Alark Eguing,False
309,9234_01,0,False,G/1500/P,0,49.0,0,6.0,1.0,638.0,1107.0,23.0,Catha Bushins,False


In [25]:
spaceship_cleaned_3=spaceship_cleaned_2.dropna()
spaceship_cleaned_3

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0002_01,0,False,F/0/S,0,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
1,0004_01,0,False,F/1/S,0,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
2,0008_03,1,False,B/1/P,1,45.0,0,39.0,7295.0,589.0,110.0,124.0,Wezena Flatic,True
3,0011_01,0,False,F/2/P,0,28.0,0,8.0,974.0,12.0,2.0,7.0,Elle Bertsontry,True
4,0056_03,1,False,A/1/S,0,27.0,0,279.0,605.0,3374.0,286.0,3.0,Zinoces Tractive,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,9196_02,1,False,C/339/S,0,31.0,0,1.0,104.0,1338.0,698.0,126.0,Majorab Poustive,True
307,9196_03,1,False,C/339/S,0,23.0,0,381.0,794.0,52.0,654.0,1317.0,Chellus Poustive,False
308,9202_01,1,False,C/309/P,0,26.0,0,8586.0,77.0,114.0,29.0,4.0,Alark Eguing,False
309,9234_01,0,False,G/1500/P,0,49.0,0,6.0,1.0,638.0,1107.0,23.0,Catha Bushins,False


In [27]:
spaceship_cleaned_3

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0002_01,0,False,F/0/S,0,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
1,0004_01,0,False,F/1/S,0,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
2,0008_03,1,False,B/1/P,1,45.0,0,39.0,7295.0,589.0,110.0,124.0,Wezena Flatic,True
3,0011_01,0,False,F/2/P,0,28.0,0,8.0,974.0,12.0,2.0,7.0,Elle Bertsontry,True
4,0056_03,1,False,A/1/S,0,27.0,0,279.0,605.0,3374.0,286.0,3.0,Zinoces Tractive,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,9196_02,1,False,C/339/S,0,31.0,0,1.0,104.0,1338.0,698.0,126.0,Majorab Poustive,True
307,9196_03,1,False,C/339/S,0,23.0,0,381.0,794.0,52.0,654.0,1317.0,Chellus Poustive,False
308,9202_01,1,False,C/309/P,0,26.0,0,8586.0,77.0,114.0,29.0,4.0,Alark Eguing,False
309,9234_01,0,False,G/1500/P,0,49.0,0,6.0,1.0,638.0,1107.0,23.0,Catha Bushins,False


In [31]:
import pandas as pd

# Load the provided CSV file

data = spaceship_cleaned_3

# Convert the 'HomePlanet', 'Destination', and 'VIP' columns into dummy/one-hot encoded variables
data_dummies = pd.get_dummies(data, columns=['HomePlanet', 'Destination', 'VIP'])

# Display the first few rows of the transformed dataframe
data_dummies

data_dummies.reset_index(drop=True, inplace=True)

transported = data_dummies.pop('Transported')
data_dummies['Transported'] = transported


data_dummies


,PassengerId,CryoSleep,Cabin,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,...,HomePlanet_2,HomePlanet_3,Destination_0,Destination_1,Destination_2,Destination_3,VIP_0,VIP_1,VIP_2,Transported
0,0002_01,False,F/0/S,24.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,...,False,False,True,False,False,False,True,False,False,True
1,0004_01,False,F/1/S,16.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,...,False,False,True,False,False,False,True,False,False,True
2,0008_03,False,B/1/P,45.0,39.0,7295.0,589.0,110.0,124.0,Wezena Flatic,...,False,False,False,True,False,False,True,False,False,True
3,0011_01,False,F/2/P,28.0,8.0,974.0,12.0,2.0,7.0,Elle Bertsontry,...,False,False,True,False,False,False,True,False,False,True
4,0056_03,False,A/1/S,27.0,279.0,605.0,3374.0,286.0,3.0,Zinoces Tractive,...,False,False,True,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,9196_02,False,C/339/S,31.0,1.0,104.0,1338.0,698.0,126.0,Majorab Poustive,...,False,False,True,False,False,False,True,False,False,True
227,9196_03,False,C/339/S,23.0,381.0,794.0,52.0,654.0,1317.0,Chellus Poustive,...,False,False,True,False,False,False,True,False,False,False
228,9202_01,False,C/309/P,26.0,8586.0,77.0,114.0,29.0,4.0,Alark Eguing,...,False,False,True,False,False,False,True,False,False,False
229,9234_01,False,G/1500/P,49.0,6.0,1.0,638.0,1107.0,23.0,Catha Bushins,...,False,False,True,False,False,False,True,False,False,False


In [33]:
data_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    231 non-null    object 
 1   CryoSleep      231 non-null    object 
 2   Cabin          231 non-null    object 
 3   Age            231 non-null    float64
 4   RoomService    231 non-null    float64
 5   FoodCourt      231 non-null    float64
 6   ShoppingMall   231 non-null    float64
 7   Spa            231 non-null    float64
 8   VRDeck         231 non-null    float64
 9   Name           231 non-null    object 
 10  HomePlanet_0   231 non-null    bool   
 11  HomePlanet_1   231 non-null    bool   
 12  HomePlanet_2   231 non-null    bool   
 13  HomePlanet_3   231 non-null    bool   
 14  Destination_0  231 non-null    bool   
 15  Destination_1  231 non-null    bool   
 16  Destination_2  231 non-null    bool   
 17  Destination_3  231 non-null    bool   
 18  VIP_0     

In [34]:
data_dummies['Transported'] = data_dummies['Transported'].astype(str)

In [37]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
for column in data_dummies.select_dtypes(include='bool').columns:
    if column != 'Transported':
        data_dummies[column] = data_dummies[column].astype(int)

# Move the 'Transported' column to the end
transported = data_dummies.pop('Transported')
data_dummies['Transported'] = transported

# Display the first few rows of the transformed dataframe
data_dummies

,PassengerId,CryoSleep,Cabin,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,...,HomePlanet_2,HomePlanet_3,Destination_0,Destination_1,Destination_2,Destination_3,VIP_0,VIP_1,VIP_2,Transported
0,0002_01,False,F/0/S,24.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,...,0,0,1,0,0,0,1,0,0,True
1,0004_01,False,F/1/S,16.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,...,0,0,1,0,0,0,1,0,0,True
2,0008_03,False,B/1/P,45.0,39.0,7295.0,589.0,110.0,124.0,Wezena Flatic,...,0,0,0,1,0,0,1,0,0,True
3,0011_01,False,F/2/P,28.0,8.0,974.0,12.0,2.0,7.0,Elle Bertsontry,...,0,0,1,0,0,0,1,0,0,True
4,0056_03,False,A/1/S,27.0,279.0,605.0,3374.0,286.0,3.0,Zinoces Tractive,...,0,0,1,0,0,0,1,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,9196_02,False,C/339/S,31.0,1.0,104.0,1338.0,698.0,126.0,Majorab Poustive,...,0,0,1,0,0,0,1,0,0,True
227,9196_03,False,C/339/S,23.0,381.0,794.0,52.0,654.0,1317.0,Chellus Poustive,...,0,0,1,0,0,0,1,0,0,False
228,9202_01,False,C/309/P,26.0,8586.0,77.0,114.0,29.0,4.0,Alark Eguing,...,0,0,1,0,0,0,1,0,0,False
229,9234_01,False,G/1500/P,49.0,6.0,1.0,638.0,1107.0,23.0,Catha Bushins,...,0,0,1,0,0,0,1,0,0,False


In [53]:
data_dummies.nunique()

PassengerId      231
CryoSleep          1
Cabin            226
Age               50
RoomService      169
FoodCourt        192
ShoppingMall     157
Spa              169
VRDeck           167
Name             231
HomePlanet_0       2
HomePlanet_1       2
HomePlanet_2       2
HomePlanet_3       2
Destination_0      2
Destination_1      2
Destination_2      2
Destination_3      2
VIP_0              2
VIP_1              2
VIP_2              2
Transported        2
dtype: int64

In [39]:
#djjjjjjjjjjdwdjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj
#wdwdwdwdwdwdwdwd
#dwdwdwdwdwdwdwdwdw
#wdwdwdwwdwdwd

In [51]:
# Select only numeric columns
numeric_spaceship = data_dummies.select_dtypes(include=['number'])

# Ensure that the 'Transported' column is excluded from the features
features = numeric_spaceship.drop(columns=["Transported"], errors='ignore')

# Extract the 'Transported' column as the target
target = data_dummies["Transported"]

# Display the first few rows of the features and target
features

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

pred = knn.predict(X_test)
pred

knn.score(X_test, y_test)

0.7446808510638298

In [44]:
features

,PassengerId,CryoSleep,Cabin,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,...,HomePlanet_1,HomePlanet_2,HomePlanet_3,Destination_0,Destination_1,Destination_2,Destination_3,VIP_0,VIP_1,VIP_2
0,0002_01,False,F/0/S,24.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,...,0,0,0,1,0,0,0,1,0,0
1,0004_01,False,F/1/S,16.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,...,0,0,0,1,0,0,0,1,0,0
2,0008_03,False,B/1/P,45.0,39.0,7295.0,589.0,110.0,124.0,Wezena Flatic,...,1,0,0,0,1,0,0,1,0,0
3,0011_01,False,F/2/P,28.0,8.0,974.0,12.0,2.0,7.0,Elle Bertsontry,...,0,0,0,1,0,0,0,1,0,0
4,0056_03,False,A/1/S,27.0,279.0,605.0,3374.0,286.0,3.0,Zinoces Tractive,...,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,9196_02,False,C/339/S,31.0,1.0,104.0,1338.0,698.0,126.0,Majorab Poustive,...,1,0,0,1,0,0,0,1,0,0
227,9196_03,False,C/339/S,23.0,381.0,794.0,52.0,654.0,1317.0,Chellus Poustive,...,1,0,0,1,0,0,0,1,0,0
228,9202_01,False,C/309/P,26.0,8586.0,77.0,114.0,29.0,4.0,Alark Eguing,...,1,0,0,1,0,0,0,1,0,0
229,9234_01,False,G/1500/P,49.0,6.0,1.0,638.0,1107.0,23.0,Catha Bushins,...,0,0,0,1,0,0,0,1,0,0


**Perform Train Test Split**

In [56]:
# Select only numeric columns
numeric_spaceship = data_dummies.select_dtypes(include=['number'])

# Ensure that the 'Transported' column is excluded from the features
features = numeric_spaceship.drop(columns=["Transported"], errors='ignore')

# Extract the 'Transported' column as the target
target = data_dummies["Transported"]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

pred = knn.predict(X_test)
pred

knn.score(X_test, y_test)

0.7446808510638298

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Assuming data_dummies is already defined and processed as per previous steps

# Select only numeric columns
numeric_spaceship = data_dummies.select_dtypes(include=['number'])

# Ensure that the 'Transported' column is excluded from the features
features = numeric_spaceship.drop(columns=["Transported"], errors='ignore')

# Extract the 'Transported' column as the target
target = data_dummies["Transported"]

# Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

# Bagging
bagging_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100,
    bootstrap=True,
    n_jobs=-1,
    random_state=0
)
bagging_clf.fit(X_train, y_train)
y_pred_bagging = bagging_clf.predict(X_test)
bagging_accuracy = accuracy_score(y_test, y_pred_bagging)

# Pasting
pasting_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100,
    bootstrap=False,
    n_jobs=-1,
    random_state=0
)
pasting_clf.fit(X_train, y_train)
y_pred_pasting = pasting_clf.predict(X_test)
pasting_accuracy = accuracy_score(y_test, y_pred_pasting)

# Display the accuracy results
bagging_accuracy, pasting_accuracy


c:\Users\ostor\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\ostor\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


(0.8297872340425532, 0.7021276595744681)

- Random Forests

In [59]:
#your code here
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming data_dummies is already defined and processed as per previous steps

# Select only numeric columns
numeric_spaceship = data_dummies.select_dtypes(include=['number'])

# Ensure that the 'Transported' column is excluded from the features
features = numeric_spaceship.drop(columns=["Transported"], errors='ignore')

# Extract the 'Transported' column as the target
target = data_dummies["Transported"]

# Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

# Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
rf_accuracy = accuracy_score(y_test, y_pred_rf)

# Display the accuracy result
rf_accuracy


0.7446808510638298

- Gradient Boosting

In [60]:
#your code here

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Assuming data_dummies is already defined and processed as per previous steps

# Select only numeric columns
numeric_spaceship = data_dummies.select_dtypes(include=['number'])

# Ensure that the 'Transported' column is excluded from the features
features = numeric_spaceship.drop(columns=["Transported"], errors='ignore')

# Extract the 'Transported' column as the target
target = data_dummies["Transported"]

# Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

# Gradient Boosting
gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=0)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, y_pred_gb)

# Display the accuracy result
gb_accuracy


0.7021276595744681

- Adaptive Boosting

In [61]:
#your code here
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

# Assuming data_dummies is already defined and processed as per previous steps

# Select only numeric columns
numeric_spaceship = data_dummies.select_dtypes(include=['number'])

# Ensure that the 'Transported' column is excluded from the features
features = numeric_spaceship.drop(columns=["Transported"], errors='ignore')

# Extract the 'Transported' column as the target
target = data_dummies["Transported"]

# Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

# Adaptive Boosting
ab_clf = AdaBoostClassifier(n_estimators=100, random_state=0)
ab_clf.fit(X_train, y_train)
y_pred_ab = ab_clf.predict(X_test)
ab_accuracy = accuracy_score(y_test, y_pred_ab)

# Display the accuracy result
ab_accuracy


0.5957446808510638

Which model is the best and why?

In [ ]:
#comment here